# Python Bug Exterminator — Week 2 Exercise

**Building on Week 1's Bug Review Panel**, this exercise creates a full Gradio-powered chat interface for analyzing buggy Python code — with streaming, multi-model support, and multi-modal output.

**Features:**
- Streaming chat UI with markdown-rendered analysis
- Three AI model variants via OpenRouter (OpenAI, Google, Anthropic)
- Three expertise levels (Junior, Intermediate, Senior SWE)
- Tool use: `check_syntax` validates code via `ast.parse()` before analysis; `log_bug` records each bug to a SQLite database
- Audio playback of analysis metadata (level, description, bug info) — code is excluded from narration
- Downloadable syntax-highlighted snapshot image of the corrected code
- Settings locked during generation to prevent mid-stream changes

In [ ]:
# If you get "No module named pip", run this in your terminal first:
#   python -m ensurepip --upgrade
# Then restart the kernel and re-run this cell.

%pip install -q gTTS Pygments Pillow

In [ ]:
import os
import re
import ast
import json
import sqlite3
import tempfile
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from gtts import gTTS
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import ImageFormatter

In [ ]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key found — add your OpenRouter key to .env as OPENAI_API_KEY")
else:
    print(f"API key loaded ({api_key[:8]}...)")

OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
client = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=api_key)

In [ ]:
MODELS = {
    "OpenAI (GPT-4o Mini)": "openai/gpt-4o-mini",
    "Google (Gemini 2.5 Flash Lite)": "google/gemini-2.5-flash-lite",
    "Anthropic (Claude 3.5 Haiku)": "anthropic/claude-3.5-haiku",
}

EXPERTISE_LEVELS = ["Junior SWE", "Intermediate SWE", "Senior SWE"]

GUARDRAIL = (
    "You are the Python Bug Exterminator. You help users with:\n"
    "1) Analyzing buggy Python code — use the structured format when doing so.\n"
    "2) Follow-up conversations about bugs, fixes, Python concepts, or code patterns.\n"
    "3) Answering questions about bug history — use the get_bug_history tool when the user "
    "asks about past bugs, statistics, or previously logged bugs.\n"
    "You should be conversational and helpful for anything related to Python, bugs, or code.\n"
    "Only redirect the user if the topic is completely unrelated to programming "
    "(e.g., sports, weather, recipes): "
    "'I'm focused on Python bug analysis — paste some code or ask about a previous fix!'\n\n"
)

SYSTEM_PROMPTS = {
    "Junior SWE": (
        GUARDRAIL
        + "You are a Junior Software Engineer with about 1 year of experience. "
        "You focus on obvious syntax errors but might miss subtle logical bugs or edge cases."
    ),
    "Intermediate SWE": (
        GUARDRAIL
        + "You are an Intermediate Software Engineer with 3-5 years of experience. "
        "You catch both syntax and logical errors and suggest practical improvements."
    ),
    "Senior SWE": (
        GUARDRAIL
        + "You are a Senior Software Engineer with 10+ years of experience. "
        "You catch all bugs — syntax, logical, and edge cases — and also consider "
        "performance, readability, naming conventions, and best practices."
    ),
}

EXAMPLE_SAMPLES = [
    [
        "def find_min(arr)\n"
        "  min = arr[0]\n"
        " for i in range(1, len(arr):\n"
        "    if arr[i]  min:\n"
        "       min = arr[i]\n"
        "  return min"
    ],
    [
        "def bubbl_sort(arra):\n"
        "  n = len(arra)\n"
        "  for i in range(n):\n"
        "    for j in range(n - i - 1)\n"
        "      if arra[j > arra[j + 1]:\n"
        "        arra[j], arra[j + 1]  arra[j + 1], arra[j]\n"
        "\n"
        "  retur arra"
    ],
    [
        "def permutations(string):\n"
        " result=[]\n"
        " if len(string)==1:\n"
        "  reurn [string]\n"
        "  for i in range(len(string):\n"
        "   char=string[i]\n"
        "   remaining=string[:1]+string[i+1]\n"
        "   for permutation in permutations(remaining)\n"
        "    result.append(char+permutation)\n"
        " return result"
    ],
]

FORMAT_INSTRUCTIONS = """
When the user submits code for analysis, you MUST respond using ONLY this exact 
template with NO extra text before, after, or in between sections. 
Absolutely no notes, tips, summaries, or commentary.

## Analysis
- **Level**: [Easy / Medium / Hard]
- **Description**: [One sentence on what the code tries to do]
- **Buggy**: [Yes / No]
- **Bug**: [Type, nature, and description of bug(s) — or "None" if not buggy]

If and ONLY if the code is buggy (Buggy: Yes), also include these two sections:

## Buggy Code
```python
[original code as provided]
```

## Corrected Code
```python
[fixed version of the code]
```

If the code is NOT buggy (Buggy: No), your response ends after the Analysis section. 
Do NOT include Buggy Code or Corrected Code sections.
"""

In [ ]:
BUGS_DB = "bugs.db"

with sqlite3.connect(BUGS_DB) as conn:
    conn.execute("""
        CREATE TABLE IF NOT EXISTS bugs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            level TEXT,
            bug_type TEXT,
            description TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()

check_syntax_function = {
    "name": "check_syntax",
    "description": "Check if the given Python code has syntax errors. Call this FIRST before writing your analysis.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "The Python code to check for syntax errors",
            },
        },
        "required": ["code"],
        "additionalProperties": False,
    },
}

log_bug_function = {
    "name": "log_bug",
    "description": "Log a discovered bug to the tracking database. Call this for each bug you identify before writing your final response.",
    "parameters": {
        "type": "object",
        "properties": {
            "level": {
                "type": "string",
                "enum": ["Easy", "Medium", "Hard"],
                "description": "Difficulty level of the code containing the bug",
            },
            "bug_type": {
                "type": "string",
                "description": "The type/category of the bug (e.g., SyntaxError, LogicError, TypeError)",
            },
            "description": {
                "type": "string",
                "description": "Brief description of the bug",
            },
        },
        "required": ["level", "bug_type", "description"],
        "additionalProperties": False,
    },
}

get_bug_history_function = {
    "name": "get_bug_history",
    "description": "Query the bug tracking database. Call this when the user asks about past bugs, bug statistics, or bug history.",
    "parameters": {
        "type": "object",
        "properties": {
            "bug_type_filter": {
                "type": "string",
                "description": "Optional filter by bug type (e.g., 'SyntaxError'). Leave empty for all bugs.",
            },
            "limit": {
                "type": "integer",
                "description": "Max number of results to return. Defaults to 10.",
            },
        },
        "required": [],
        "additionalProperties": False,
    },
}

tools = [
    {"type": "function", "function": check_syntax_function},
    {"type": "function", "function": log_bug_function},
    {"type": "function", "function": get_bug_history_function},
]

TOOL_INSTRUCTIONS = (
    "You have access to three tools:\n"
    "1. check_syntax — Call this FIRST with the user's code to verify syntax errors.\n"
    "2. log_bug — After identifying bugs, call this to log each bug to the tracking database.\n"
    "3. get_bug_history — Call this when the user asks about past bugs, statistics, or history.\n"
    "Always use check_syntax before writing your analysis. "
    "Always use log_bug for each bug you find before writing your final response.\n\n"
)

In [ ]:
def extract_code_blocks(text):
    """Extract the first two Python code blocks from the response (buggy and corrected)."""
    matches = re.findall(r'```python\s*(.*?)\s*```', text, re.DOTALL)
    buggy = matches[0].strip() if len(matches) > 0 else ""
    corrected = matches[1].strip() if len(matches) > 1 else ""
    return buggy, corrected


def clean_for_speech(text):
    """Strip markdown formatting so TTS reads natural English, not 'backtick colon backtick'."""
    text = re.sub(r'`([^`]*)`', r'\1', text)
    text = re.sub(r'\*\*([^*]*)\*\*', r'\1', text)
    text = re.sub(r'\*([^*]*)\*', r'\1', text)
    text = re.sub(r'[_#>]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def extract_metadata(text):
    """Extract analysis metadata for audio narration (excludes code)."""
    fields = {
        "Level": r'\*\*Level\*\*:\s*(.+)',
        "Description": r'\*\*Description\*\*:\s*(.+)',
        "Buggy": r'\*\*Buggy\*\*:\s*(.+)',
        "Bug": r'\*\*Bug\*\*:\s*(.+)',
    }
    parts = []
    for label, pattern in fields.items():
        match = re.search(pattern, text)
        if match:
            parts.append(f"{label}: {clean_for_speech(match.group(1).strip())}")
    return ". ".join(parts) if parts else None


def generate_audio(metadata_text):
    """Generate an MP3 narration of the metadata using gTTS."""
    if not metadata_text:
        return None
    try:
        tts = gTTS(text=metadata_text, lang='en')
        tmp = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
        tts.save(tmp.name)
        return tmp.name
    except Exception:
        return None


def generate_code_image(code):
    """Generate a syntax-highlighted PNG snapshot of the corrected code."""
    if not code:
        return None
    try:
        formatter = ImageFormatter(
            style='monokai',
            font_size=14,
            line_numbers=True,
            line_pad=6,
            image_pad=12,
        )
        img_data = highlight(code, PythonLexer(), formatter)
        tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
        tmp.write(img_data)
        tmp.close()
        return tmp.name
    except Exception:
        return None


def get_last_response(history):
    """Get the content of the last assistant message."""
    if history and history[-1]["role"] == "assistant":
        return history[-1]["content"]
    return ""


def check_syntax(code):
    """Check Python code for syntax errors using ast.parse()."""
    try:
        ast.parse(code)
        return "No syntax errors found. The code parses successfully."
    except SyntaxError as e:
        return f"Syntax error on line {e.lineno}: {e.msg}"


def log_bug(level, bug_type, description):
    """Log a discovered bug to the SQLite tracking database."""
    with sqlite3.connect(BUGS_DB) as conn:
        conn.execute(
            "INSERT INTO bugs (level, bug_type, description) VALUES (?, ?, ?)",
            (level, bug_type, description),
        )
        conn.commit()
    return f"Bug logged: [{level}] {bug_type} — {description}"


def get_bug_history(bug_type_filter="", limit=10):
    """Query the bug tracking database for past bugs."""
    with sqlite3.connect(BUGS_DB) as conn:
        if bug_type_filter:
            rows = conn.execute(
                "SELECT id, level, bug_type, description, timestamp FROM bugs "
                "WHERE bug_type LIKE ? ORDER BY timestamp DESC LIMIT ?",
                (f"%{bug_type_filter}%", limit),
            ).fetchall()
        else:
            rows = conn.execute(
                "SELECT id, level, bug_type, description, timestamp FROM bugs "
                "ORDER BY timestamp DESC LIMIT ?",
                (limit,),
            ).fetchall()

    if not rows:
        return "No bugs found in the database yet."

    total = conn.execute("SELECT COUNT(*) FROM bugs").fetchone()[0]
    lines = [f"Showing {len(rows)} of {total} total bugs:\n"]
    for row in rows:
        lines.append(f"- #{row[0]} [{row[1]}] {row[2]}: {row[3]} ({row[4]})")
    return "\n".join(lines)


def handle_tool_calls(message):
    """Dispatch tool calls from the LLM and return results."""
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "check_syntax":
            result = check_syntax(arguments["code"])
        elif tool_call.function.name == "log_bug":
            result = log_bug(
                arguments["level"],
                arguments["bug_type"],
                arguments["description"],
            )
        elif tool_call.function.name == "get_bug_history":
            result = get_bug_history(
                arguments.get("bug_type_filter", ""),
                arguments.get("limit", 10),
            )
        else:
            result = "Unknown tool"
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id,
        })
    return responses

In [ ]:
def on_submit(message, history):
    """Add user message to chat and clear the input box."""
    if not message.strip():
        return gr.update(), history
    history = history + [{"role": "user", "content": message}]
    return "", history


def disable_controls():
    """Lock settings and submit button during generation."""
    return (
        gr.update(interactive=False),
        gr.update(interactive=False),
        gr.update(interactive=False),
    )


def stream_response(history, model_name, expertise):
    """Handle tool calls behind the scenes, then stream the final analysis."""
    if not history or history[-1]["role"] != "user":
        yield history
        return

    system = SYSTEM_PROMPTS[expertise] + TOOL_INSTRUCTIONS + FORMAT_INSTRUCTIONS
    model = MODELS[model_name]

    messages = [{"role": "system", "content": system}]
    messages += [{"role": h["role"], "content": h["content"]} for h in history]

    # Phase 1: Non-streaming tool call loop
    response = client.chat.completions.create(model=model, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        msg = response.choices[0].message
        tool_responses = handle_tool_calls(msg)
        messages.append(msg)
        messages.extend(tool_responses)
        response = client.chat.completions.create(model=model, messages=messages, tools=tools)

    # Phase 2: Stream the final response using enriched context
    stream = client.chat.completions.create(model=model, messages=messages, stream=True)

    history = history + [{"role": "assistant", "content": ""}]
    for chunk in stream:
        history[-1]["content"] += chunk.choices[0].delta.content or ""
        yield history


def post_process(history):
    """Re-enable controls and reset audio after generation completes."""
    return (
        gr.update(interactive=True),
        gr.update(interactive=True),
        gr.update(interactive=True),
        gr.update(interactive=True),
        None,
    )


def on_play(history):
    """Generate audio narration of the analysis metadata (not the code)."""
    response = get_last_response(history)
    metadata = extract_metadata(response)
    return generate_audio(metadata)


def disable_play():
    """Disable play button after audio is generated."""
    return gr.update(interactive=False)


def on_download(history):
    """Generate a downloadable syntax-highlighted image of the corrected code."""
    response = get_last_response(history)
    _, corrected = extract_code_blocks(response)
    return generate_code_image(corrected)

In [ ]:
with gr.Blocks(title="Python Bug Exterminator", theme=gr.themes.Soft()) as app:
    gr.Markdown(
        "# Python Bug Exterminator\n"
        "*Paste buggy Python code, pick your AI model and expertise level, then hit Analyze.*"
    )

    chatbot = gr.Chatbot(height=420, type="messages", label="Analysis")

    with gr.Row():
        play_btn = gr.Button("Play Analysis Audio", variant="secondary")
        download_btn = gr.Button("Download Code Snapshot", variant="secondary")

    with gr.Row():
        audio_output = gr.Audio(label="Analysis Audio", autoplay=False)
        image_output = gr.File(label="Code Snapshot")

    with gr.Row():
        model_dd = gr.Dropdown(
            choices=list(MODELS.keys()),
            value=list(MODELS.keys())[0],
            label="AI Model Variant",
            interactive=True,
        )
        level_dd = gr.Dropdown(
            choices=EXPERTISE_LEVELS,
            value=EXPERTISE_LEVELS[0],
            label="Expertise Level",
            interactive=True,
        )

    with gr.Row():
        code_input = gr.Textbox(
            label="Paste your buggy Python code",
            placeholder="def find_min(arr)\n  min = arr[0]\n  ...",
            lines=6,
        )
        submit_btn = gr.Button("Send", variant="primary", scale=0)

    gr.Examples(
        examples=EXAMPLE_SAMPLES,
        inputs=[code_input],
        label="Try a buggy sample from Week 1",
    )

    # Event chain: add message -> lock settings -> stream response -> extract code & unlock
    submit_btn.click(
        on_submit, [code_input, chatbot], [code_input, chatbot]
    ).then(
        disable_controls, None, [model_dd, level_dd, submit_btn]
    ).then(
        stream_response, [chatbot, model_dd, level_dd], [chatbot]
    ).then(
        post_process, [chatbot],
        [model_dd, level_dd, submit_btn, play_btn, audio_output]
    )

    play_btn.click(
        on_play, [chatbot], [audio_output]
    ).then(
        disable_play, None, [play_btn]
    )
    download_btn.click(on_download, [chatbot], [image_output])

app.launch()